In [1]:
!pip install openai
!pip install git+https://github.com/theskumar/python-dotenv.git

  Cloning https://github.com/theskumar/python-dotenv.git to /tmp/pip-req-build-0_tmzprw
  Running command git clone --filter=blob:none --quiet https://github.com/theskumar/python-dotenv.git /tmp/pip-req-build-0_tmzprw
  Resolved https://github.com/theskumar/python-dotenv.git to commit d6c0b9638349a7dd605d60ee555ff60421c1a594
  Preparing metadata (setup.py) ... done


In [2]:
from dotenv import load_dotenv
import os
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

# Cargar variables de entorno
load_dotenv()
movies_data = pd.read_csv("/content/drive/MyDrive/dataIA/movies/movies_with_languagesRevised.csv")
movies_data.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,title,genres_associated,languages_availables,runtime,overview,adult,release_date,tagline,vote_average,keywords,cast
0,Jumanji,"['Adventure', 'Fantasy', 'Family']","['English', 'French']",104.0,When siblings Judy and Peter discover an encha...,False,1995-12-15,Roll the dice and unleash the excitement!,6.9,"['board game', 'disappearance', ""based on chil...","['Robin Williams', 'Jonathan Hyde', 'Kirsten D..."
1,Grumpier Old Men,"['Romance', 'Comedy']",['English'],101.0,A family wedding reignites the ancient feud be...,False,1995-12-22,Still Yelling. Still Fighting. Still Ready for...,6.5,"['fishing', 'best friend', 'duringcreditssting...","['Walter Matthau', 'Jack Lemmon', 'Ann-Margret..."
2,Waiting to Exhale,"['Comedy', 'Drama', 'Romance']",['English'],127.0,"Cheated on, mistreated and stepped on, the wom...",False,1995-12-22,Friends are the people who let you be yourself...,6.1,"['based on novel', 'interracial relationship',...","['Whitney Houston', 'Angela Bassett', 'Loretta..."
3,Father of the Bride Part II,['Comedy'],['English'],106.0,Just when George Banks has recovered from his ...,False,1995-02-10,Just When His World Is Back To Normal... He's ...,5.7,"['baby', 'midlife crisis', 'confidence', 'agin...","['Steve Martin', 'Diane Keaton', 'Martin Short..."
4,Heat,"['Action', 'Crime', 'Drama', 'Thriller']","['English', 'Spanish']",170.0,"Obsessive master thief, Neil McCauley leads a ...",False,1995-12-15,A Los Angeles Crime Saga,7.7,"['robbery', 'detective', 'bank', 'obsession', ...","['Al Pacino', 'Robert De Niro', 'Val Kilmer', ..."


In [3]:
from openai import OpenAI
import re

# Configura tu clave de API
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

counter = 0
# Función para obtener la calificación mínima de edad utilizando OpenAI GPT-3
def obtener_calificacion_minima(title, overview):
    # Concatenar la información de entrada en un solo texto
    # prompt = f"La película {title}, lanzada en {release_date}, es una película que trata sobre {overview}. Según su contenido, ¿qué edad mínima crees que sería apropiada?"
    prompt = f"¿Para qué edad ves recomendada la película {title} que trata sobre {overview}? Devuelveme sólo la edad, si es para todos los publicos devuelve solo un 0"
    counter = 0

    # Hacer la consulta a GPT
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": f"{prompt}"}
        ],    max_tokens=100  # Aumenta el número de tokens máximos
        )
    try:
        if counter % 100 == 0:
            print("----------------------------------------------------------------------------------------------------------------------------------------------------------")
        if response.choices:
            counter += 1
            choice = response.choices[0]
            if choice and choice.message.content and not choice.message.content.startswith("No puedo proporcionar"):
                print("choices: ", choice.message.content)

                # Buscar la edad en el mensaje utilizando una expresión regular
                match = re.search(r'\b\d+\b', choice.message.content)
                if match:
                    print(match.group())
                    return int(match.group())
    except Exception as e:
        # Manejar cualquier excepción que pueda ocurrir
        print(f"Error al obtener la calificación mínima de edad: {e}")
    return None

In [4]:
movies_data['calificacion'] = movies_data.head(5000).apply(lambda row: obtener_calificacion_minima(row['title'], row['overview']), axis=1)
movies_data.head()['calificacion']

----------------------------------------------------------------------------------------------------------------------------------------------------------
choices:  La película Jumanji se recomienda para todas las edades, por lo que se devuelve un 0.
0
----------------------------------------------------------------------------------------------------------------------------------------------------------
choices:  La película "Grumpier Old Men" está recomendada para todas las edades, por lo tanto, la edad sería 0.
0
----------------------------------------------------------------------------------------------------------------------------------------------------------
choices:  La película "Waiting to Exhale" no es apta para todos los públicos y no se recomienda para menores de 13 años. 
13
----------------------------------------------------------------------------------------------------------------------------------------------------------
choices:  La película Father of the Bride P

KeyboardInterrupt: 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Suponiendo que ya tienes movies_data como un DataFrame de pandas con la columna "calificacion" que contiene la edad recomendada

# Cuenta cuántas películas hay para cada edad recomendada
calificacion_counts = movies_data.head(5000)['calificacion'].value_counts().sort_index()

# Crea la gráfica
plt.figure(figsize=(10, 6))
calificacion_counts.plot(kind='bar', color='skyblue')
plt.title('Cantidad de Películas por Edad Recomendada')
plt.xlabel('Edad Recomendada')
plt.ylabel('Cantidad de Películas')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Muestra la gráfica
plt.tight_layout()
plt.show()

In [1]:
# Guardar el DataFrame actualizado
movies_data.to_csv("/content/drive/MyDrive/dataIA/movies/movies_with_recommendedAge.csv", index=False)

NameError: name 'movies_data' is not defined